In [ ]:
from conjecture.consolidated_functions import *
from conjecture.all_valid_orderings import *
from conjecture.optimization_functions import *
from iterative_heuristic.modified_etf import Mod_ETF
from iterative_heuristic.approximate_speeds import *
from makespan_energy.construct_graph_util import *
from makespan_energy.visualization_util import *
import networkx as nx
import numpy as np
import math

In [ ]:
# Objective is to minimize MRT + E
mrt = True
tie_breaking_rule = 2
plot = False
counter = 0
max_counter = 100
save_seed = None

while counter != max_counter:
    counter += 1
    print("Test " + str(counter) )
    # Random
    num_tasks = 10
    num_machines = 10
    edges = 10
    seed=random.randint(0, 100000)
    
    G = random_dag(num_tasks, edges, seed)
    print(seed)
#     make_graph_visual(G, num_tasks)
    
    w = [1 for _ in range(num_tasks)]
    s = [1 for i in range(num_tasks)]

    # Get ordering using modified ETF
    test = Mod_ETF(G, w, s, num_machines, tie_breaking_rule, plot=plot)

    # Initialize objective function value
    heuristic_opt = test.obj_value

    # Get pseudosize, convert to speed
    psize, num_shared_task_lst = approx_psize_homogeneous(G, test.order, test.h, test.t, verbose=False)
    s_prime = psize_to_speed(psize)

    # Get ordering using modified ETF
    test2 = Mod_ETF(G, w, s_prime, num_machines, tie_breaking_rule, plot=plot)


    # Check if the objective function value has improved
    if heuristic_opt > test2.obj_value:
        heuristic_opt = test2.obj_value
        s = s_prime

    print("Final objective value: " + str(heuristic_opt))

    # Given ordering, find optimal speeds
    print("From opt check:")
    temp = get_objective_single_ordering(mrt, G, w, test.order, plot=plot, compare=False)
    opt_intervals, s_opt, obj_opt, _ = temp 
    if s_opt[0] != -1:
        #get upper bound for each task
        ub = ub_lst(G, test.order)
       
        for i in range(num_tasks):
            
            print(float(np.sqrt(ub[i])).__round__(5) / s_opt[i])

            if float(np.sqrt(ub[i])).__round__(5) / s_opt[i] < 1:
                for j in range(num_tasks):
                    print("task " + str(j) + ": " + str(np.sqrt(ub[j]).__round__(5)) + "/" + str(s_opt[j]))
                print("STOP")

                counter = max_counter
                save_seed = seed
                break

    else:
        print("SKIP THIS ONE")

In [ ]:
plot = True

max_overall_ratio = - np.infty


seed=save_seed
G = random_dag(num_tasks, edges, seed)
print(seed)


w = [1 for _ in range(num_tasks)]
s = [1 for i in range(num_tasks)]

# Get ordering using modified ETF
test = Mod_ETF(G, w, s, num_machines, tie_breaking_rule, plot=plot)

# Initialize objective function value
heuristic_opt = test.obj_value

# Get pseudosize, convert to speed
psize, num_shared_task_lst = approx_psize_homogeneous(G, test.order, test.h, test.t, verbose=True)
s_prime = psize_to_speed(psize)

# Get ordering using modified ETF
test2 = Mod_ETF(G, w, s_prime, num_machines, tie_breaking_rule, plot=plot)


# Check if the objective function value has improved
if heuristic_opt > test2.obj_value:
    heuristic_opt = test2.obj_value
    s = s_prime

print("Final objective value: " + str(heuristic_opt))

# Given ordering, find optimal speeds
print("From opt check:")
temp = get_objective_single_ordering(mrt, G, w, test.order, plot=plot, compare=False)
opt_intervals, s_opt, obj_opt, _ = temp 
if s_opt[0] != -1:
    ub = ub_lst(G, test.order)
    
    
 
   
    for i in range(num_tasks):
      
        if float(np.sqrt(ub[i])).__round__(5) / s_opt[i] < 1:
            
            for j in range(num_tasks):
                print("task " + str(j) + ": " + str(np.sqrt(ub[j]).__round__(5)) + "/" + str(s_opt[j]) + ", " + str(float(np.sqrt(ub[j])).__round__(5) / s_opt[j]))
            print("STOP")
            make_graph_visual(G, num_tasks)
            break


else:
    print("SKIP THIS ONE")